In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import shap
DATASET = 'datasets/diamonds/diamonds.csv'

# The Dataset
First let's describe the dataset to find any errors

In [ ]:
dataset = pd.read_csv(DATASET)
dataset.describe()

In [ ]:
dataset['cut'].value_counts()

In [ ]:
dataset['color'].value_counts()

In [ ]:
dataset['clarity'].value_counts()

We can see that there are gems with negative price and x,y,z with size 0. First we will clean all inconsistent data

In [ ]:
dataset.drop(dataset[dataset.price <= 0].index, inplace=True)
dataset.drop(dataset[dataset.x <= 0].index, inplace=True)
dataset.drop(dataset[dataset.y <= 0].index, inplace=True)
dataset.drop(dataset[dataset.z <= 0].index, inplace=True)
dataset.describe()

Then we encode color, cut and clarity to a numeric value

In [ ]:
dataset['cut'] = dataset['cut'].map({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})
dataset['color'] = dataset['color'].map({'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6})
dataset['clarity'] = dataset['clarity'].map({'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7})
dataset.head()

In [ ]:
dataset.boxplot(column='price')
plt.show()

There is a high number of price outliers!

In [ ]:
plt.figure(figsize=(10, 8))
p = sns.heatmap(dataset.corr(), annot=True)
plt.show()

The dataset's README and a quick Google search says that the most fundamental properties of a diamond are the `carat`, `cut`, `color` and `clarity`, but the correlation matrix shows `cut`, `color` and `clarity` have a _negative_ correlation with `price` and that the properties with the highest correlation to `price` are the `carat` and the `x,y,z` size! My guess is that the `carat` rules the price, even if it has a not so good `cut`, `color` and `clarity`. As the `carat` represents the weight, it is natural that these are bigger volume wise in the `x,y,z` dimensions, so that's why both have a high correlation to price.

## Model

We'll do a classic 80/20 train test data split and then run some classic regression models using the `sklearn` library. We'll keep the one with the highest score. As there is a high number of outliers we will use `RobustScaler` for the target which has a high tolerance for outliers, and `StandardScaler` for the other columns.

We'll be dropping `depth` as it has a very low correlation with `price`. `x,y,z` will also be dropped due to their high correlation with `carat`. This way the model will be simpler and, therefore, easier to explain.

In [ ]:
X = dataset.drop(columns=['price', 'depth', 'x', 'y', 'z'])
#X['volume'] = dataset['x'] * dataset['y'] * dataset['z']

In [ ]:
y = dataset['price']
# Split the dataset into train and test sets. Use random_state=42 so that the results are reproducible.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_test.shape

In [ ]:
# Classic linear regression
model = TransformedTargetRegressor(regressor=LinearRegression(), transformer=RobustScaler()).fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
# Lasso regression
model = TransformedTargetRegressor(regressor=Lasso(random_state=42), transformer=RobustScaler()).fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
# SVM regression
model = TransformedTargetRegressor(regressor=SVR(), transformer=RobustScaler()).fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
# Random Forest regression
model = TransformedTargetRegressor(regressor=RandomForestRegressor(random_state=42), transformer=RobustScaler()).fit(X_train, y_train)
model.score(X_test, y_test)

## Results

It seems that the best regression models for this dataset are the ones based on Support Vector Machines and Random Forests. Some cross validations tests can be made to be more sure of which of these two to choose, but both would do a pretty good job with an $R^2$ score of ~0.97. More robust solutions using feature engineering, further data cleaning, using a better decision tree algorithm such as _XGBoost_ and/or neural networks could improve the accuracy even further.

## Explainability

To explain the Random Forest regression we can check the `feature_importances` as specified here https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

Here we see that features with a high MDI in the forest have a high impact on the final result of the regression.

In [ ]:
# Graph feature_importances_ of the model
std = np.std([tree.feature_importances_ for tree in model.regressor_.estimators_], axis=0)
forest_importances = pd.Series(model.regressor_.feature_importances_, index=X.columns)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.show()

As expected, `carat` is the clearest `price` indicator. Sadly, there is no easy way of getting this information in a Support Vector Machine model. There is a well known AI explainability library called `SHAP` that can help us in this case and which can be used for other models we may implement.

(Takes around 5 minutes to run)

In [ ]:
model = TransformedTargetRegressor(regressor=SVR(), transformer=RobustScaler()).fit(X_train, y_train)
explainer = shap.KernelExplainer(model.predict, X_train)
data = shap.sample(X_test, 100)
shap_values = explainer.shap_values(data)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", feature_names=X.columns)

As expected, the features relevance are quite similar to its `price` correlation on the correlation matrix. These graphs can be shown to Don Francesco to explain both models results.